# Multiple Agent Chat

### Requirement

Auto Agent setup need `vllm>=0.2.0` and `fastchat>=0.2.30`, you can use the following command to install the latest version of [vLLM](https://github.com/vllm-project/vllm) and [FastChat](https://github.com/lm-sys/FastChat).
```
pip install vllm, "fastchat[model_worker]"
```

In [ ]:
!pip install vllm "fastchat[model_worker]"

In [1]:
import autogen
import subprocess as sp
import time
from typing import *

In [2]:
class AgentCreator:
    """
    Descriptions
    """

    agent_procs: Dict[str, sp.Popen] = {}
    agent_procs_assign: Dict[str, Tuple[autogen.AssistantAgent, str]] = {}
    openai_server_name: str = 'openai'

    def __init__(
            self,
            endpoint_building_timeout: Optional[int] = 180
    ):
        """
        Args:
            endpoint_building_timeout: timeout for building up an endpoint server.
        """
        self.endpoint_building_timeout = endpoint_building_timeout

    def build_endpoint(
            self,
            model_name_or_hf_repo: str,
            world_size: Optional[int] = 1,
            host: Optional[str] = "127.0.0.1",
            port: Optional[str] = "8000",
            config_path: Optional[str] = './OAI_CONFIG_LIST'
    ) -> Tuple[dict, str]:
        """
        Descriptions

        Args:
            model_name_or_hf_repo: model name (e.g., "gpt-3.5-turbo", "gpt-4") or huggingface repository
                (e.g., "meta-llama/Llama-2-7b-chat-hf") use for starting an endpoint.
            world_size: the max size of parallel tensors (in most of the cases, this is identical to the amount of GPUs).
            host: the endpoint's API address.
            port: the port of the API address the endpoint listen to.
            config_path: config list path of OpenAI api.

        Returns:
            (agent config list, endpoint server id)
        """
        config_list = autogen.config_list_from_json(
            config_path,
            filter_dict={
                'model': [model_name_or_hf_repo]
            }
        )
        if "gpt-" in model_name_or_hf_repo:
            server_id = self.openai_server_name
        else:
            model_name = model_name_or_hf_repo.split('/')[-1]
            config_list[0]['api_base'] = f"http://{host}:{port}/v1"
            server_id = f'{model_name}_{host}_{port}'
            if self.agent_procs.get(server_id, None) is None:
                # Use vLLM to set up a server with OpenAI API support.
                agent_proc = sp.Popen(['python', '-m', 'vllm.entrypoints.openai.api_server',
                                       f'--host', f'{host}',
                                       f'--port', f'{port}',
                                       f'--model', f'{model_name_or_hf_repo}',
                                       f'--tensor-parallel-size', f'{world_size}'], stdout=sp.PIPE, stderr=sp.STDOUT)
                timeout_start = time.time()
                while True:
                    server_stdout = agent_proc.stdout.readline()
                    timeout_end = time.time()
                    if b"running" in server_stdout:
                        print(f"Running {model_name_or_hf_repo} on http://{host}:{port} with tensor parallel size {world_size}.")
                        break
                    elif timeout_end - timeout_start > self.endpoint_building_timeout:
                        raise RuntimeError(f'Timeout exceed. Fail to set up the endpoint for '
                                           f'{model_name_or_hf_repo} on {host}:{port}.')
                self.agent_procs[server_id] = agent_proc

        return config_list, server_id

    def create_agent(
            self,
            agent_name: str,
            llm_config: dict,
            server_id: str,
            **kwargs
    ) -> autogen.AssistantAgent:
        """
        Descriptions

        Args:
            agent_name: the name that identify the function of the agent (e.g., Coder, Product Manager,...)
            llm_config: specific configs for LLM (e.g., config_list, seed, temperature, ...).
            server_id: endpoint server id.

        Returns:
            agent: a set-up agent.
        """
        agent = autogen.AssistantAgent(name=agent_name,
                                       llm_config=llm_config,
                                       **kwargs)
        self.agent_procs_assign[agent_name] = (agent, server_id)
        return agent

    def clear_agent(
            self,
            agent_name: str = None,
            recycle_endpoint: bool = True
    ):
        """
        Descriptions

        Args:
            agent_name: the name of agent.
            recycle_endpoint: trigger for recycle the endpoint server. If true, the endpoint will be recycled
                when there is no agent depending on.
        """
        _, server_id = self.agent_procs_assign[agent_name]
        del self.agent_procs_assign[agent_name]
        if recycle_endpoint:
            if server_id == self.openai_server_name:
                return
            else:
                for _, iter_sid in self.agent_procs_assign.values():
                    if server_id == iter_sid:
                        return
                self.agent_procs[server_id].terminate()
    
    def clear_all(self):
        """
        Clear all cached agents.
        """
        for agent_name in [agent_name for agent_name in self.agent_procs_assign.keys()]:
            self.clear_agent(agent_name)

    def start(
            self,
            task: str,
            manager_llm_config: dict,
            manager_server_id: str,
            max_round: Optional[int] = 12,
            init_messages: Optional[List[dict]] = [],
            user_proxy: autogen.UserProxyAgent = None,
            initiate_agent_name: str = "user"
    ):
        """
        Descriptions

        Args:
            task: task description.
            manager_llm_config: LLM configuration for a GroupChatManager.
            manager_server_id: id of the server the manager depending on.
            max_round: the maximum number of rounds.
            init_messages: input messages before the task start. This can be the chat history from other group chat
                or some preliminary of the task.
            user_proxy: a user proxy for human interaction.
            initiate_agent_name: the name of an agent use to initialize the group chat.
        """
        agent_list = [agent for agent, _ in self.agent_procs_assign.values()]
        if user_proxy is not None:
            agent_list.append(user_proxy)
        group_chat = autogen.GroupChat(agents=agent_list, messages=init_messages, max_round=max_round)
        manager = autogen.GroupChatManager(groupchat=group_chat, llm_config=manager_llm_config)
        self.agent_procs_assign['manager'] = (manager, manager_server_id)

        if initiate_agent_name == "user" and user_proxy is not None:
            user_proxy.initiate_chat(manager, message=task)
        else:
            for agent in agent_list:
                if initiate_agent_name == agent.name():
                    agent.initiate_chat(manager, message=task)
        
        self.clear_agent('manager')

# Test

In [3]:
administrator = AgentCreator()
config_path = '/home/elpis_ubuntu/LLM/autogen/OAI_CONFIG_LIST'

# build endpoint; create agent; get config (manager); initialize user proxy; task start; clear agent
gpt3_5_config_list, gpt3_5_server_id = administrator.build_endpoint('gpt-3.5-turbo', config_path=config_path)
opt_config_list, opt_server_id = administrator.build_endpoint('facebook/opt-125m', config_path=config_path)

coder_llm_config = {
    'config_list': gpt3_5_config_list,
    'temperature': 0,
}
pm_llm_config = {
    'config_list': opt_config_list,
    'temperature': 0,
    'max_tokens': 200
}
mg_llm_config = {
    'config_list': gpt3_5_config_list,
    'temperature': 0,
}

administrator.create_agent('Coder_gpt-3_5', coder_llm_config, gpt3_5_server_id)
administrator.create_agent('Product_manager_opt-125m', pm_llm_config, opt_server_id)

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
    human_input_mode="TERMINATE"
)

task = "Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
administrator.start(
    task,
    mg_llm_config,
    gpt3_5_server_id,
    user_proxy=user_proxy,
    initiate_agent_name="user"
)
administrator.clear_all()


Running facebook/opt-125m on http://127.0.0.1:8000 with tensor parallel size 1.
User_proxy (to chat_manager):

Find a latest paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------
Coder_gpt-3_5 (to chat_manager):

To find the latest paper about GPT-4 on arXiv, we can use the arXiv API to search for papers related to GPT-4. Then, we can analyze the paper to identify its potential applications in software.

Here is the Python code to accomplish this task:

```python
import requests

# Search for papers related to GPT-4 on arXiv
search_query = "GPT-4"
api_url = f"http://export.arxiv.org/api/query?search_query={search_query}&sortBy=submittedDate&sortOrder=descending&max_results=1"
response = requests.get(api_url)

# Parse the XML response and extract the paper details
xml_data = response.text
paper_title = xml_data.split("<title>")[2].split("</title>")[0]
paper_summary = xml_data.split("<

KeyboardInterrupt: 

In [3]:
administrator = AgentCreator()
config_path = '/home/elpis_ubuntu/LLM/autogen/OAI_CONFIG_LIST'

# build endpoint; create agent; get config (manager); initialize user proxy; task start; clear agent
gpt3_5_config_list, gpt3_5_server_id = administrator.build_endpoint('gpt-3.5-turbo', config_path=config_path)
gpt4_config_list, gpt4_server_id = administrator.build_endpoint('gpt-4', config_path=config_path)

coder_llm_config = {
    'config_list': gpt3_5_config_list,
    'temperature': 0,
}
pm_llm_config = {
    'config_list': gpt4_config_list,
    'temperature': 0,
}
mg_llm_config = {
    'config_list': gpt3_5_config_list,
    'temperature': 0,
}

administrator.create_agent('Coder_gpt-3_5', coder_llm_config, gpt3_5_server_id)
administrator.create_agent('Product_manager_gpt-4', pm_llm_config, gpt4_server_id)
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
    human_input_mode="TERMINATE"
)

task = "Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
administrator.start(
    task,
    mg_llm_config,
    gpt3_5_server_id,
    user_proxy=user_proxy,
    initiate_agent_name="user"
)
administrator.clear_all()

User_proxy (to chat_manager):

Find a latest paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------
Coder_gpt-3_5 (to chat_manager):

To find the latest paper about GPT-4 on arXiv, we can use the arXiv API to search for papers related to GPT-4. Then, we can analyze the paper to identify its potential applications in software.

Here is the Python code to accomplish this task:

```python
import requests

# Search for papers related to GPT-4 on arXiv
search_query = "GPT-4"
api_url = f"http://export.arxiv.org/api/query?search_query={search_query}&sortBy=submittedDate&sortOrder=descending&max_results=1"
response = requests.get(api_url)

# Parse the XML response and extract the paper details
xml_data = response.text
paper_title = xml_data.split("<title>")[2].split("</title>")[0]
paper_summary = xml_data.split("<summary>")[1].split("</summary>")[0]

# Print the paper details
print("Latest Pa

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


User_proxy (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 10, in <module>
    paper_title = xml_data.split("<title>")[2].split("</title>")[0]
IndexError: list index out of range


--------------------------------------------------------------------------------
Coder_gpt-3_5 (to chat_manager):

I apologize for the error. It seems that the code I provided to extract the paper details from the XML response is not working as expected. Let's try an alternative approach using the `xml.etree.ElementTree` module to parse the XML response.

Please use the updated code below:

```python
import requests
import xml.etree.ElementTree as ET

# Search for papers related to GPT-4 on arXiv
search_query = "GPT-4"
api_url = f"http://export.arxiv.org/api/query?search_query={search_query}&sortBy=submittedDate&sortOrder=descending&max_results=1"
response = requests.get(api_url)

# Parse the XML response and extract the paper details
root =

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


User_proxy (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
Latest Paper on GPT-4:
Title: Skill-Mix: a Flexible and Expandable Family of Evaluations for AI models
Summary:   With LLMs shifting their role from statistical modeling of language to
serving as general-purpose AI agents, how should LLM evaluations change?
Arguably, a key ability of an AI agent is to flexibly combine, as needed, the
basic skills it has learned. The capability to combine skills plays an
important role in (human) pedagogy and also in a paper on emergence phenomena
(Arora & Goyal, 2023).
  This work introduces Skill-Mix, a new evaluation to measure ability to
combine skills. Using a list of $N$ skills the evaluator repeatedly picks
random subsets of $k$ skills and asks the LLM to produce text combining that
subset of skills. Since the number of subsets grows like $N^k$, for even modest
$k$ this evaluation will, with high probability, require the LLM to produce
text significantly different from